# Working with the annoation framework
#### As proofreading occurs, the segment IDs associated with each objevt will be dynamically updated each time an edit is made. In order to keep track of neurons (or other objects) of interest, we can instead associate annotations with spatial points that exist within our neuron, and then update the segment ID associated with this spatial point as edits are made.

#### Our current implementation of the annotation framework does not have a functioning materialization engine. To work around this, we can use the `annotations` module of `FANC_auto_recon` to upload and download annotation tables for analysis. This notebook will outline how to do this. 

- This example will demonstrate how to take a neuroglancer state where we annotated a series of cell bodies from hemilineage 7B, and upload those annoatations to the annotation framework hemilineage table. The workflow would be as follows:
    1. Generate an annotation layer in your neuroglancer instance named '7B'.
    2. Add a series of point annotations in each of the soma (not in the nucleus! as that is often segmented separately from the rest of the neuron. 
    3. Save the json state using `shift-cnrl-j`
    4. Retreive and parse the json state using `FANC_auto_recon.annotations`
    5. Upload the new entries to the annotation table.
    


#### For more information:

- The annoatation framework for FANC is housed here: https://github.com/ZettaAI/helm-charts/wiki/Annotation-Framework-Client
- It is described here: https://annotationframeworkclient.readthedocs.io/en/latest/


In [2]:
from FANC_auto_recon.annotations import schema_upload
from FANC_auto_recon.segmentation import authentication_utils
from FANC_auto_recon.annotations import schema_download

#### Get an annotation framework client object which will allow interaction with the framework.

In [3]:
client,token = authentication_utils.get_client()

##### The primary method for interacting with neuroglancer is via parsing of json states. Specify the json state id and the name of the annotation layer from which to obtain the point annotations.

In [4]:
json_id = 103789102624132678
layer_name = '7B'
table_name = 'Test_Table'
description = 'A table for learning about the annotation framework'

state = client.state.get_state_json(json_id)

#### Generate formatted entries for the annotation framework `bount-tag` schema and upload them to the annotation framework. Don't do this more than once for the same set of data. You can remove it, but that requires other steps. 

In [ ]:
soma_table = schema_upload.soma_table_entries(state,layer_name=layer_name,description=layer_name)
schema_upload.upload_cells(client,soma_table,table_name,description=description)

## Now that we have uploaded data, let's retreive it. Here is the workflow:
1. Download the raw annotation table.
2. 'Materialize' the data (look up the most recent root IDs associated with the tagged points. This step relies on having `~/.cloudvolume/segmentations` set up. See FANC_auto_recon documentation for how to do this. 

In [5]:
raw_table = schema_download.download_annotation_table(client,table_name)
materialized_table = schema_download.generate_soma_table(raw_table,
                                                     segmentation_version='FANC_production_segmentation')

Segmentation IDs: 100%|██████████| 50/50 [00:07<00:00,  6.60it/s]


In [6]:
materialized_table.head()

,name,cell_type,pt_position,pt_root_id,soma_x_nm,soma_y_nm,soma_z_nm,found
0,7B,NaN,"[31253, 91038, 2652]",648518346494145512,134387.9,391463.4,119340.0,NaN
1,7B,NaN,"[29841, 91260, 2656]",648518346516310999,128316.3,392418.0,119520.0,NaN
2,7B,NaN,"[29076, 93551, 2656]",648518346485958242,125026.8,402269.3,119520.0,NaN
3,7B,NaN,"[27178, 93145, 2656]",648518346492889628,116865.4,400523.5,119520.0,NaN
4,7B,NaN,"[27976, 93766, 2618]",648518346493573774,120296.8,403193.8,117810.0,NaN
